In [ ]:
%run parsing.ipynb
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# https://networkx.github.io/documentation/latest/tutorial.html#edges
class WeightGraph:
    def __init__(self):
        self.g = nx.Graph()
        
    def add(self, a, b, delta):
        self.g.add_node(a)
        self.g.add_node(b)
        new_value = self.get(a, b) + delta
        self.g.add_edge(a, b, weight=new_value)
        
    def get(self, a, b):
        if b in self.g.adj[a]:
            return self.g.adj[a][b]["weight"]
        return 0

In [ ]:
class RepoFile:
    def __init__(self, repo, path):
        self.repo = repo
        self.path = path
        self.content = None
        self.tree = None
    
    def get_content(self):
        if self.content is None:
            self.content = self.repo.get_file_content(self.path)
        return self.content
    
    def get_tree(self):
        if self.tree is None:
            self.tree = java_parser.parse(self.get_content())
        return self.tree
    
    def node_text(self, node):
        return self.content[node.start_byte:node.end_byte].decode("utf-8")
    
    def walk_tree(self, node_handler):
        """ node_handler gets the current logic-path and node for each ast node"""
        self.walk_tree_cursor(self.get_tree().walk(), self.path, node_handler)
    
    def walk_tree_cursor(self, cursor, prefix, node_handler):
        if not cursor.node.is_named:
            return
        node_handler(prefix, cursor.node)
        # cursor.current_field_name() is the role that this node has in its parent
        tree_node_name = None
        if cursor.node.type == "class_declaration":
            idfield = cursor.node.child_by_field_name("name")
            tree_node_name = self.node_text(idfield)
        elif cursor.node.type == "field_declaration":
            idfield = cursor.node.child_by_field_name("declarator").child_by_field_name("name")
            tree_node_name = self.node_text(idfield)
        elif cursor.node.type == "method_declaration":
            idfield = cursor.node.child_by_field_name("name")
            tree_node_name = self.node_text(idfield)

        if tree_node_name is not None:
            prefix = prefix + "/" + tree_node_name
            # found_nodes.register(prefix)

        if cursor.goto_first_child():
            self.walk_tree_cursor(cursor, prefix, node_handler)
            while cursor.goto_next_sibling():
                self.walk_tree_cursor(cursor, prefix, node_handler)
            cursor.goto_parent()
